In [1]:
import os
import random
import shutil
from pathlib import Path
import json
import math
import pandas as pd

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from PIL import Image
from torchvision import transforms

from kymatio.torch import Scattering2D
import pywt
from collections import defaultdict
import clip
from transformers import AutoImageProcessor, AutoModel, AutoProcessor, AutoModelForImageClassification

from scipy.special import softmax

d:\IPTP\Stage_3A\Stage\Code\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from encoder import Encoder
from wst2vit import clip_encoder, dinov2_encoder
import time
from rpo import SparseProjector, GaussianProjector
import utils
from eval import Metrics
from padim import mahalanobis_detector, padim_detector

In [3]:
from patchcore import greedy_coreset_selection, coreset_detector, patchcore_detector

In [4]:
with open("classes.json", "r", encoding="utf-8") as f:
    data = json.load(f)
classes_idx = data["1k_idx"]
classes_names = data["21k_idx"]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Encoding 

In [6]:
clip_encoder = Encoder("clip")
for cls in classes_idx:
    for generator in ["bgan", "midj", "sd_15", "nature", "nature_2"]:
        input_path = f"../Data/GenImage/{cls}/{generator}"
        save_path_clip = f"../Data/Features/clip/{cls}/{generator}.pt"
        os.makedirs(os.path.dirname(save_path_clip), exist_ok=True)
        tensor = clip_encoder(input_path)
        torch.save(tensor, save_path_clip)


Clip Direct Encoding:   0%|          | 0/2 [00:00<?, ?it/s]d:\IPTP\Stage_3A\Stage\Code\.venv\lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
Clip Direct Encoding: 100%|██████████| 2/2 [00:01<00:00,  1.17it/s]


torch.Size([162, 512])


Clip Direct Encoding: 100%|██████████| 2/2 [00:06<00:00,  3.21s/it]


torch.Size([162, 512])


Clip Direct Encoding: 100%|██████████| 2/2 [00:03<00:00,  1.56s/it]


...

torch.Size([162, 512])


Clip Direct Encoding: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


torch.Size([162, 512])


Clip Direct Encoding: 100%|██████████| 2/2 [00:01<00:00,  1.05it/s]

torch.Size([162, 512])


In [7]:
# clip_patch_encoder = Encoder("clip", patchify = True)
dinov2_patch_encoder = Encoder("dinov2", patchify = True)
# tc = {"clip": [], "dinov2": []}
for cls in classes_idx:
    for generator in ["bgan", "midj", "sd_15", "nature", "nature_2"]:
        input_path = f"../Data/GenImage/{cls}/{generator}"
        # save_path_clip = f"../Data/Features/clip_patch/{cls}/{generator}.pt"
        save_path_dinov2 = f"../Data/Features/dinov2_patch/{cls}/{generator}.pt"
        # os.makedirs(os.path.dirname(save_path_clip), exist_ok=True)
        os.makedirs(os.path.dirname(save_path_dinov2), exist_ok=True)
        # start = time.time()
        # tensor1 = clip_patch_encoder(input_path)
        # end = time.time()
        # tc["clip"].append(float(end - start))
        # torch.save(tensor1, save_path_clip)
        # start = time.time()
        tensor2 = dinov2_patch_encoder(input_path)
        # end = time.time()
        # tc["dinov2"].append(float(end - start))
        torch.save(tensor2, save_path_dinov2)



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Dinov2 Patches Encoding:   0%|          | 0/2 [00:00<?, ?it/s]d:\IPTP\Stage_3A\Stage\Code\.venv\lib\site-packages\transformers\integrations\sdpa_attention.py:81: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Dinov2 Patches Encoding: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]


torch.Size([162, 256, 768])


Dinov2 Patches Encoding: 100%|██████████| 2/2 [00:07<00:00,  3.67s/it]


torch.Size([162, 256, 768])


Dinov2 Patches Encoding: 100%|██████████| 2/2 [00:03<00:00,  1.76s/it]


...

torch.Size([162, 256, 768])


Dinov2 Patches Encoding: 100%|██████████| 2/2 [00:02<00:00,  1.35s/it]


torch.Size([162, 256, 768])


Dinov2 Patches Encoding: 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]

torch.Size([162, 256, 768])


In [6]:
clip_time = np.mean(tc["clip"])
dinov2_time = np.mean(tc["dinov2"])


In [7]:
print(clip_time, dinov2_time)

3.2608294677734375 3.895852737426758


In [11]:
dinov2_mean_encoder = Encoder("dinov2")
tc = {"dinov2": []}
for cls in classes_idx:
    for generator in ["bgan", "midj", "sd_15", "nature", "nature_2"]:
        input_path = f"../Data/GenImage/{cls}/{generator}"
        save_path_dinov2 = f"../Data/Features/dinov2_mean/{cls}/{generator}.pt"
        os.makedirs(os.path.dirname(save_path_dinov2), exist_ok=True)
        start = time.time()
        tensor2 = dinov2_mean_encoder(input_path)
        end = time.time()
        tc["dinov2"].append(float(end - start))
        torch.save(tensor2, save_path_dinov2)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Dinov2 Direct Encoding: 100%|██████████| 2/2 [00:03<00:00,  1.66s/it]


torch.Size([162, 768])


Dinov2 Direct Encoding: 100%|██████████| 2/2 [00:08<00:00,  4.45s/it]


torch.Size([162, 768])


Dinov2 Direct Encoding: 100%|██████████| 2/2 [00:04<00:00,  2.36s/it]


...

torch.Size([162, 768])


Dinov2 Direct Encoding: 100%|██████████| 2/2 [00:03<00:00,  1.78s/it]


torch.Size([162, 768])


Dinov2 Direct Encoding: 100%|██████████| 2/2 [00:03<00:00,  1.86s/it]

torch.Size([162, 768])


In [12]:
print(np.mean(tc["dinov2"]))

4.94347270488739


In [8]:
wst2_encoder = Encoder("wst", J=2)
wst3_encoder = Encoder("wst", J=3)
tc = {"wst2": [], "wst3": []}
for cls in classes_idx:
    for generator in ["bgan", "midj", "sd_15", "nature", "nature_2"]:
        input_path = f"../Data/GenImage/{cls}/{generator}"
        save_path_wst2 = f"../Data/Features/wst2/{cls}/{generator}.pt"
        save_path_wst3 = f"../Data/Features/wst3/{cls}/{generator}.pt"
        os.makedirs(os.path.dirname(save_path_wst2), exist_ok=True)
        os.makedirs(os.path.dirname(save_path_wst3), exist_ok=True)
        start = time.time()
        tensor1 = wst2_encoder(input_path)
        end = time.time()
        tc["wst2"].append(float(end - start))
        torch.save(tensor1, save_path_wst2)
        start = time.time()
        tensor2 = wst3_encoder(input_path)
        end = time.time()
        tc["wst3"].append(float(end - start))
        torch.save(tensor2, save_path_wst3)



WST-2 Direct Encoding: 100%|██████████| 2/2 [00:01<00:00,  1.90it/s]


torch.Size([162, 81, 64, 64])


WST-3 Direct Encoding: 100%|██████████| 2/2 [00:00<00:00,  2.24it/s]


torch.Size([162, 217, 32, 32])


WST-2 Direct Encoding: 100%|██████████| 2/2 [00:04<00:00,  2.35s/it]


...

torch.Size([162, 217, 32, 32])


WST-2 Direct Encoding: 100%|██████████| 2/2 [00:01<00:00,  1.94it/s]


torch.Size([162, 81, 64, 64])


WST-3 Direct Encoding: 100%|██████████| 2/2 [00:01<00:00,  1.68it/s]


torch.Size([162, 217, 32, 32])


In [9]:
print(np.mean(tc["wst2"]), np.mean(tc["wst3"]))

2.0060737657547 2.12442666053772


## Clip, Dinov2-mean recalculation with Mahalanobis

In [7]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    gt_tensor = torch.load(f"../Data/Features/clip/{cls}/nature.pt",weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/clip/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/clip/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = mahalanobis_detector(gt_tensor, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/clip_mahalanobis.csv", index=False)
print(f"clip mahalanobis auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

Mahalanobis scoring:: 100%|██████████| 324/324 [00:00<00:00, 535.22it/s]

clip mahalanobis auroc: 0.5141365645480872, auprc: 0.512197574958008, fpr95: 0.8090534979423866


In [10]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    gt_tensor = torch.load(f"../Data/Features/dinov2_mean/{cls}/nature.pt",weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/dinov2_mean/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/dinov2_mean/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = mahalanobis_detector(gt_tensor, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/dinov2_mean_mahalanobis.csv", index=False)
print(f"dinov2 mean mahalanobis auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

Mahalanobis scoring:: 100%|██████████| 324/324 [00:00<00:00, 459.85it/s]

dinov2 mean mahalanobis auroc: 0.6954757912919779, auprc: 0.6718992831746406, fpr95: 0.6034979423868312


## Clip, Dinov2, WST-3 recalculation with Memory Coreset and PatchCore

### Memory Coreset

In [ ]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    gt_tensor = torch.load(f"../Data/Features/clip/{cls}/nature.pt",weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/clip/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    memory_coreset, m_idx = greedy_coreset_selection(gt_tensor, l=0.5)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/clip/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = coreset_detector(memory_coreset, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/clip_coreset.csv", index=False)
print(f"clip coreset auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

clip coreset auroc: 0.5391962607326118, auprc: 0.532540669358274, fpr95: 0.8164609053497943


In [9]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    gt_tensor = torch.load(f"../Data/Features/dinov2_mean/{cls}/nature.pt",weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/dinov2_mean/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    memory_coreset, m_idx = greedy_coreset_selection(gt_tensor, l=0.5)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/dinov2_mean/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = coreset_detector(memory_coreset, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/dinov2_mean_coreset.csv", index=False)
print(f"dinov2 mean coreset auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

dinov2 mean coreset auroc: 0.7117309353248997, auprc: 0.6806247266903488, fpr95: 0.6123456790123457


In [10]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    proj_s = SparseProjector(217, 16)
    gt_tensor = torch.load(f"../Data/Features/wst3/{cls}/nature.pt", weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/wst3/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_tensor = torch.nn.functional.adaptive_avg_pool2d(gt_tensor, output_size=(16, 16))
    real_tensor = torch.nn.functional.adaptive_avg_pool2d(real_tensor, output_size=(16, 16))
    gt_tensor = utils.wst_m(gt_tensor, proj_s)
    real_tensor = utils.wst_m(real_tensor, proj_s)
    memory_coreset, m_idx = greedy_coreset_selection(gt_tensor, l=0.5)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/wst3/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        fake_tensor = torch.nn.functional.adaptive_avg_pool2d(fake_tensor, output_size=(16, 16))
        fake_tensor = utils.wst_m(fake_tensor, proj_s)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = coreset_detector(memory_coreset, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/wst3_sparse_coreset.csv", index=False)
print(f"wst3 sparse coreset auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

wst3 sparse coreset auroc: 0.44994157394706086, auprc: 0.4743008738162199, fpr95: 0.9211934156378601


### PatchCore

In [ ]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
projector = SparseProjector(768,100)
for cls in classes_idx:
    gt_patches = torch.load(f"../Data/Features/clip_patch/{cls}/nature.pt",weights_only = True).to(DEVICE)
    real_patches = torch.load(f"../Data/Features/clip_patch/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_patches = gt_patches.reshape(-1, gt_patches.shape[-1])
    gt_patches_proj = projector.project(gt_patches)
    coreset, indices = greedy_coreset_selection(gt_patches_proj, l=0.5)
    memory_coreset = gt_patches[indices]
    for generator in ["bgan", "midj", "sd_15"]:
        fake_patches = torch.load(f"../Data/Features/clip_patch/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        labels = np.concatenate((np.zeros(real_patches.shape[0]), np.ones(fake_patches.shape[0])))
        scores = patchcore_detector(memory_coreset, real_patches, fake_patches)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/clip_patchcore.csv", index=False)
print(f"clip patchcore auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

clip patchcore auroc: 0.38810839302951794, auprc: 0.42072097432556643, fpr95: 0.9390946502057613


In [12]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
projector = SparseProjector(768,100)
for cls in classes_idx:
    gt_patches = torch.load(f"../Data/Features/dinov2_patch/{cls}/nature.pt",weights_only = True).to(DEVICE)
    real_patches = torch.load(f"../Data/Features/dinov2_patch/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_patches = gt_patches.reshape(-1, gt_patches.shape[-1])
    gt_patches_proj = projector.project(gt_patches)
    coreset, indices = greedy_coreset_selection(gt_patches_proj, l=0.5)
    memory_coreset = gt_patches[indices]
    for generator in ["bgan", "midj", "sd_15"]:
        fake_patches = torch.load(f"../Data/Features/dinov2_patch/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        labels = np.concatenate((np.zeros(real_patches.shape[0]), np.ones(fake_patches.shape[0])))
        scores = patchcore_detector(memory_coreset, real_patches, fake_patches)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/dinov2_patchcore.csv", index=False)
print(f"dinov2 patchcore auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

dinov2 patchcore auroc: 0.5911281308743586, auprc: 0.5762045610478104, fpr95: 0.7818930041152262


## WST 2&3 -Mahalanobis

In [ ]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    proj_s = SparseProjector(81, 8)
    gt_tensor = torch.load(f"../Data/Features/wst2/{cls}/nature.pt", weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/wst2/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_tensor = torch.nn.functional.adaptive_avg_pool2d(gt_tensor, output_size=(16, 16))
    real_tensor = torch.nn.functional.adaptive_avg_pool2d(real_tensor, output_size=(16, 16))
    gt_tensor = utils.wst_m(gt_tensor, proj_s)
    real_tensor = utils.wst_m(real_tensor, proj_s)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/wst2/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        fake_tensor = torch.nn.functional.adaptive_avg_pool2d(fake_tensor, output_size=(16, 16))
        fake_tensor = utils.wst_m(fake_tensor, proj_s)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = mahalanobis_detector(gt_tensor, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/wst2_sparse_mahalanobis.csv", index=False)
print(f"wst2 mahalanobis auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

        



wst2 mahalanobis auroc: 0.4254648681603414, auprc: 0.47431262034051974, fpr95: 0.9049382716049384


In [22]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    proj_s = SparseProjector(217, 16)
    gt_tensor = torch.load(f"../Data/Features/wst3/{cls}/nature.pt", weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/wst3/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_tensor = torch.nn.functional.adaptive_avg_pool2d(gt_tensor, output_size=(16, 16))
    real_tensor = torch.nn.functional.adaptive_avg_pool2d(real_tensor, output_size=(16, 16))
    gt_tensor = utils.wst_m(gt_tensor, proj_s)
    real_tensor = utils.wst_m(real_tensor, proj_s)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/wst3/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        fake_tensor = torch.nn.functional.adaptive_avg_pool2d(fake_tensor, output_size=(16, 16))
        fake_tensor = utils.wst_m(fake_tensor, proj_s)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = mahalanobis_detector(gt_tensor, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/wst3_sparse_mahalanobis.csv", index=False)
print(f"wst3 sparse mahalanobis auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

wst3 sparse mahalanobis auroc: 0.4263742823756541, auprc: 0.48064977056659886, fpr95: 0.8965020576131687


In [23]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    proj_g = GaussianProjector(81,8)
    gt_tensor = torch.load(f"../Data/Features/wst2/{cls}/nature.pt", weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/wst2/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_tensor = torch.nn.functional.adaptive_avg_pool2d(gt_tensor, output_size=(16, 16))
    real_tensor = torch.nn.functional.adaptive_avg_pool2d(real_tensor, output_size=(16, 16))
    gt_tensor = utils.wst_m(gt_tensor, proj_g)
    real_tensor = utils.wst_m(real_tensor, proj_g)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/wst2/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        fake_tensor = torch.nn.functional.adaptive_avg_pool2d(fake_tensor, output_size=(16, 16))
        fake_tensor = utils.wst_m(fake_tensor, proj_g)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = mahalanobis_detector(gt_tensor, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/wst2_gaussian_mahalanobis.csv", index=False)
print(f"wst2 gaussian mahalanobis auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

wst2 gaussian mahalanobis auroc: 0.42860844383478136, auprc: 0.4762146218749842, fpr95: 0.905349794238683


In [24]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
for cls in classes_idx:
    proj_g = GaussianProjector(217,16)
    gt_tensor = torch.load(f"../Data/Features/wst3/{cls}/nature.pt", weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/wst3/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_tensor = torch.nn.functional.adaptive_avg_pool2d(gt_tensor, output_size=(16, 16))
    real_tensor = torch.nn.functional.adaptive_avg_pool2d(real_tensor, output_size=(16, 16))
    gt_tensor = utils.wst_m(gt_tensor, proj_g)
    real_tensor = utils.wst_m(real_tensor, proj_g)
    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/wst3/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        fake_tensor = torch.nn.functional.adaptive_avg_pool2d(fake_tensor, output_size=(16, 16))
        fake_tensor = utils.wst_m(fake_tensor, proj_g)
        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = mahalanobis_detector(gt_tensor, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/wst3_gaussian_mahalanobis.csv", index=False)
print(f"wst3 gaussian mahalanobis auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

wst3 gaussian mahalanobis auroc: 0.42946959304984006, auprc: 0.4792573234349718, fpr95: 0.8997942386831274


## WST2VIT Mahalanobis

In [8]:
dinov2_model = AutoModel.from_pretrained("facebook/dinov2-with-registers-base").to(DEVICE)
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}

for cls in classes_idx:
    proj_s = SparseProjector(81, 48)

    gt_tensor = torch.load(f"../Data/Features/wst2/{cls}/nature.pt", weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/wst2/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_tensor = utils.wst_m(gt_tensor, proj_s, False)
    real_tensor = utils.wst_m(real_tensor, proj_s, False)
    gt_tensor = utils.reshape_to_3(gt_tensor)
    real_tensor = utils.reshape_to_3(real_tensor)
    gt_tensor = dinov2_encoder(gt_tensor, dinov2_model)
    real_tensor = dinov2_encoder(real_tensor, dinov2_model)
    os.makedirs(f"../Data/Features/wst2dinov2/{cls}", exist_ok=True)
    torch.save(gt_tensor.detach().cpu(), f"../Data/Features/wst2dinov2/{cls}/nature.pt")
    torch.save(real_tensor.detach().cpu(), f"../Data/Features/wst2dinov2/{cls}/nature_2.pt")

    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/wst2/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        fake_tensor = utils.wst_m(fake_tensor, proj_s, False)
        fake_tensor = utils.reshape_to_3(fake_tensor)
        fake_tensor = dinov2_encoder(fake_tensor, dinov2_model)
        torch.save(fake_tensor.detach().cpu(), f"../Data/Features/wst2dinov2/{cls}/{generator}.pt")

        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = mahalanobis_detector(gt_tensor, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/wst2dinov2_mahalanobis.csv", index=False)
print(f"wst2dinov2 mahalanobis auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

WSTDinov2 Encoding:   0%|          | 0/3 [00:00<?, ?it/s]d:\IPTP\Stage_3A\Stage\Code\.venv\lib\site-packages\transformers\integrations\sdpa_attention.py:81: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Mahalanobis scoring:: 100%|██████████| 324/324 [00:00<00:00, 433.98it/s]

wst2dinov2 mahalanobis auroc: 0.5155438703449677, auprc: 0.530614766708069, fpr95: 0.9125514403292183


In [10]:
data = {"CLASS": [], "GENERATOR": [], "AUROC": [], "AUPRC": [], "FPR95": []}
clip_model, _ = clip.load("ViT-B/32", device=DEVICE)

for cls in classes_idx:
    proj_s = SparseProjector(81, 48)

    gt_tensor = torch.load(f"../Data/Features/wst2/{cls}/nature.pt", weights_only = True).to(DEVICE)
    real_tensor = torch.load(f"../Data/Features/wst2/{cls}/nature_2.pt", weights_only = True).to(DEVICE)
    gt_tensor = utils.wst_m(gt_tensor, proj_s, False)
    real_tensor = utils.wst_m(real_tensor, proj_s, False)
    gt_tensor = utils.reshape_to_3(gt_tensor)
    real_tensor = utils.reshape_to_3(real_tensor)
    gt_tensor = clip_encoder(gt_tensor, clip_model)
    real_tensor = clip_encoder(real_tensor, clip_model)
    os.makedirs(f"../Data/Features/wst2clip/{cls}", exist_ok=True)
    torch.save(gt_tensor.detach().cpu(), f"../Data/Features/wst2clip/{cls}/nature.pt")
    torch.save(real_tensor.detach().cpu(), f"../Data/Features/wst2clip/{cls}/nature_2.pt")

    for generator in ["bgan", "midj", "sd_15"]:
        fake_tensor = torch.load(f"../Data/Features/wst2/{cls}/{generator}.pt", weights_only = True).to(DEVICE)
        fake_tensor = utils.wst_m(fake_tensor, proj_s, False)
        fake_tensor = utils.reshape_to_3(fake_tensor)
        fake_tensor = clip_encoder(fake_tensor, clip_model)
        torch.save(fake_tensor.detach().cpu(), f"../Data/Features/wst2clip/{cls}/{generator}.pt")

        labels = np.concatenate((np.zeros(real_tensor.shape[0]), np.ones(fake_tensor.shape[0])))
        scores = mahalanobis_detector(gt_tensor, real_tensor, fake_tensor)
        m = Metrics(labels, scores)
        r,p,f = m.computation()
        data["CLASS"].append(cls)
        data["GENERATOR"].append(generator)
        data["AUROC"].append(r)
        data["AUPRC"].append(p)
        data["FPR95"].append(f)

df = pd.DataFrame(data)
df.to_csv("results_v2/wst2clip_mahalanobis.csv", index=False)
print(f"wst2clip mahalanobis auroc: {np.mean(data['AUROC'])}, auprc: {np.mean(data['AUPRC'])}, fpr95: {np.mean(data['FPR95'])}")

Mahalanobis scoring:: 100%|██████████| 324/324 [00:00<00:00, 490.09it/s]

wst2clip mahalanobis auroc: 0.5798379312096734, auprc: 0.5807227509781118, fpr95: 0.8676954732510289
